In [ ]:
""" phase_1 import modules """
#import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
""" phase_2-1 define loading functions """
def load_data():
    global Folders, Path, ID, Act_index, index_loc
    '''
    output:
    Data[i][j][k][m][n](all elements are float 64)
    
    i:  0 for phone_accel, 1 for phone_gyro
        2 for watch_accel, 3 for watch_gyro
        
    j:  index of subjects, default range between 0 and 51
    
    k:  activities
        
        0-A-walking     1-B-jogging     2-C-stairs      3-D-sitting
        4-E-standing    5-F-typing      6-G-teeth       7-H-soup
        8-I-chips       9-J-pasta       10-K-drinking   11-L-sandwich
        12-M-kicking    13-O-catch      14-P-dribbling  15-Q-writing
        16-R-calpping   17-S-folding 

    m:  0 for x_axis, 1 for y_axis, 2 for z_axis
    
    n:  index of information in time sequence
    
    (through this funciton, only sensors data are left,
    time stamps and others are confiscated)
    
    '''
    Data = []
    for i in range(len(Folders)):
        print('current folder: %s/%s' % (Folders[i][0], Folders[i][1]))
        data = []
        for j in range(len(ID)):
            array = pd.read_csv(Path % 
                          (Folders[i][0], Folders[i][1], ID[j],
                           Folders[i][1], Folders[i][0]), 
                           header=None, delimiter=',|;')
            array.drop(axis=1, columns=6, inplace=True)
            array = np.array(array)
            mat = []
            for k in range(len(Act_index)):
                pos  = [m for m,arr in enumerate(array) if arr[index_loc]==Act_index[k]]
                temp = np.array([array[m] for m in pos]).T
                temp = np.array(np.delete(temp, [0, 1, 2], axis=0), dtype=float)
                mat.append(temp)
            data.append(mat)
        Data.append(data)
    print('loading completed!')
    return Data

In [ ]:
""" phase_2-2 define window-based-dividing functions """
def split_data(Data, target=18, subj=51, sens=4, window=10, stride=5, Hz=20):
    global Folders
    ''' 
    input:
    Data  : outcome from function-load_data()
    target: numbers of classification, default = 18
    subj  : numbers of subjects, default = 51
    sens  : numbers of sensors, default = 4
    window: window of sampling length(second)
    stride: stride of sampling length(second)
    Hz    : sampling Hz, default = 20
    
    output:
    data[i][j][k][l][m][n](all elements are float 64)
    
    i, j, k, m, n are same as function2-1 'load_data()'
    
    l:  index of the piece of window in sequence
    
    '''
    data = [] 
    # data has 4 lists, 
    # each one for a combination of phone/watch and accel/gyro
    for i in range(sens):
        print('current data from sensor: %s-%s' % (Folders[i][0],Folders[i][1]))
        dat = []
        # dat has 18 lists,
        # each one for the all the dataset in one activity
        for j in range(subj):
            matrix = []
            # matrix has 51 lists
            # each one for an assemblage in one subject
            for k in range(target):
                mat = []
                array = Data[i][j][k].T
                length = len(array)
                div = (length // (stride * Hz)) - 1
                for m in range(div):
                    arr = array[(m * stride * Hz):
                                (m * stride * Hz + window * Hz)]
                    temp = []
                    for n in range(3):
                        temp.append(arr.T[n] - np.mean(arr.T[n]))
                    mat.append(np.array(temp))
                matrix.append(mat)
            dat.append(matrix)
        data.append(dat)
    print('splitting completed!')
    return data

In [ ]:
""" phase_2-3 define merging functions """
def merge_data(Data, target=18, subj=51, sens=4, shuffle=True):
    '''
    input:
    shuffle: whether shuffle or not, default = True
    
    output_1:
    data[i][j][k](all elements are float 64)
    
    i:  index of dataset, random arranged if shuffle=True
    
    j:  0, 1, 2 for x,y,z in phone_accel
        3, 4, 5 for x,y,z in phone_gyro
        6, 7, 8 for x,y,z in watch_accel
        9,10,11 for x,y,z in watch_gyro
        
    k:  default=200 pieces of information
        otherwise has window * Hz pieces
    
    output_2:
    target[i]
    
    i:  corresponding activity classification with data[i][j][k]
    
    '''
    matrix, clas = [], []
    data, target = [], []
    for k in range(18):
        for j in range(51):
            length = min(len(Data[0][j][k]),len(Data[1][j][k]),
                         len(Data[2][j][k]),len(Data[3][j][k]))
            for l in range(length):
                temp = np.concatenate((Data[0][j][k][l],
                                       Data[1][j][k][l],
                                       Data[2][j][k][l],
                                       Data[3][j][k][l]))
                matrix.append(temp)
                clas.append(k)
    if shuffle:
        index = np.random.permutation(len(clas))   
        for i in range(len(index)):
            data.append(matrix[index[i]])
            target.append(clas[index[i]]) 
    else:
        data = matrix
        target = clas
    print('merging completed')
    return data, target

In [ ]:
""" phase_2-4 define displaying functions """
def display_raw(arr, index, target, window=[0,10]):
    global Folders, axis, Act_names, Hz
    '''
    input:
    arr    : raw original data without preprocessing
             (from function2-1 'load_data' )
    index  : index of subject
    target : class of activity
    window : window for viusalising(second)
    Hz     : sampling hz, default=20
    
    output : four figs
    '''
    plt.figure(figsize=(6,12))
    plt.rcParams['figure.dpi']=120
    plt.rcParams['savefig.dpi']=120
    for i in range(4):
        plt.subplot(4,1,i+1)
        plt.subplots_adjust(hspace=0.5)
        x = np.linspace(0, len(Data[i][index][target][0])/20, 
                        len(Data[i][index][target][0]))
        plt.plot(x, Data[i][index][target][0], color=[0.1,0.3,0.7], 
                 alpha=0.9, lw=0.5, label='%s' % (axis[0]))
        plt.plot(x, Data[i][index][target][1], color=[0.7,0.3,0.1], 
                 alpha=0.9, lw=0.5, label='%s' % (axis[1]))
        plt.plot(x, Data[i][index][target][2], color=[0.1,0.7,0.3], 
                 alpha=0.9, lw=0.5, label='%s' % (axis[2]))
        plt.legend(loc="upper right", fontsize=10)
        plt.title('%s - %s, class = %s' % (Folders[i][0],Folders[i][1],
                                          Act_names[target]),fontsize=10)
        plt.xlim([window[0], window[1]])
        plt.ylim([-20, 20])
        plt.xlabel('second',fontsize=10)
        plt.grid(alpha=0.1)
    plt.show()

def display_win(arr, target, index):
    global Folders, axis, window, Hz
    '''
    input:
    arr    : raw data with preprocessing
             (from function2-3 'merge_data' )
    index  : index of data
    target : class of activity
             (from function2-3 'merge_data' )
    
    output : four figs
    
    '''
    x = np.linspace(0, window, window * Hz)
    plt.figure(figsize=(6,12))
    plt.rcParams['figure.dpi']=120
    plt.rcParams['savefig.dpi']=120
    for i in range(4):
        plt.subplot(4,1,i+1)
        plt.subplots_adjust(hspace=0.5)
        plt.plot(x, arr[index][i * 3 + 0], color=[0.1,0.3,0.7], 
                 alpha=0.7, lw=1.5, label='%s' % (axis[0]))
        plt.plot(x, arr[index][i * 3 + 1], color=[0.7,0.3,0.1], 
                 alpha=0.7, lw=1.5, label='%s' % (axis[1]))
        plt.plot(x, arr[index][i * 3 + 2], color=[0.1,0.7,0.3], 
                 alpha=0.7, lw=1.5, label='%s' % (axis[2]))
        plt.legend(loc="upper right", fontsize=10)
        plt.title('%s - %s, class = %s' % (Folders[i][0],Folders[i][1],
                                          Act_names[target[index]]),fontsize=10)
        plt.xlim([0, window])
        plt.ylim([-20, 20])
        plt.xlabel('second',fontsize=10)
        plt.grid(alpha=0.1)
    plt.show()

In [ ]:
""" phase_2-5 define dividing functions """
def divide_data(Data, target, ratio=0.1):
    length = len(Data)
    length_train = round(length * (1 - ratio))
    train_set = Data[0:(length_train-1)]
    train_tar = target[0:(length_train-1)]
    test_set = Data[length_train:-1]
    test_tar = target[length_train:-1]
    return train_set, test_set, train_tar, test_tar

In [ ]:
""" phase_3-1 build network """
class HaiNet(nn.Module):
    global window, Hz
    def __init__(self):
        super(HaiNet, self).__init__()
        self.layer_1 = nn.Sequential(
            nn.Conv1d(in_channels=12,out_channels=36,
                      kernel_size=5,stride=1,padding=2,bias=True),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))
        self.layer_2 = nn.Sequential(
            nn.Conv1d(in_channels=36,out_channels=72,
                      kernel_size=9,stride=1,padding=4,bias=True),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(72*(int((window*Hz)/4)), 720)
        self.fc2 = nn.Linear(720, 108)
        self.fc3 = nn.Linear(108, 18)          

    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [ ]:
""" phase_3-2 define loss function """
def crossloss(output, target):
    global Act_index
    length = len(Act_index)
    tar = np.zeros(length)
    tar[target-1] = 1
    tar = torch.from_numpy(tar).float()
    d = torch.dot(output, tar)
    c = - torch.log(d)
    return c

In [ ]:
""" phase_3-3 define softmax function """
def softmax(output):
    global Act_index
    out = output[0]
    length = len(Act_index)
    out_exp = torch.zeros(length)
    for i in range(length):
        out_exp[i] = torch.exp(out[i])
    out_sum = torch.sum(out_exp)
    out_prb = torch.zeros(length)
    for i in range(length):
        out_prb[i] = out_exp[i] / out_sum
    return out_prb

In [ ]:
""" phase_3-4 define correct function """
def corr(output, target):
    global Act_index
    length = len(Act_index)
    tar = np.zeros(length)
    tar[target-1] = 1
    out = output.detach().numpy()
    out_max = out[0]
    max_pos = 0
    for i in range(length):
        if out[i] > out_max:
            out_max = out[i]
            max_pos = i
    if max_pos == target - 1:
        return 1
    else:
        return 0

In [ ]:
""" phase_3-5 define train function """
def train(train_data, target):
    net.train()
    total = 0
    correct = 0
    train_loss = 0
    
    for i in range(len(train_data)):
        
        optimizer.zero_grad()
        inputs = np.reshape(train_data[i], (1,12,200))
        inputs = torch.from_numpy(inputs)
        outputs = net(inputs)
        outputs = softmax(outputs)
        loss = crossloss(outputs, target[i])
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print('%5d / %6d' % (i + 1, len(train_data)))
            acc = correct / total
            los = train_loss / total
            print('acc: %2.2f%%, loss: %1.3f\n' % (acc * 100, los))

            total = 0
            correct = 0
            train_loss = 0
            
            
        total += 1
        correct += corr(outputs, target[i])
        train_loss += loss.item()
        
    '''
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        
    return train_loss, correct, total
    '''

In [ ]:
""" phase_4 parameters """
''' 
file loading part 
(need global in function definition)

'''
Path = './raw/%s/%s/data_%d_%s_%s.txt'
Folders = (['phone','accel'],['phone','gyro'],
           ['watch','accel'],['watch','gyro'])
ID = np.linspace(1600, 1650, 51, dtype=int)
Act_index = ('A','B','C','D','E','F',
             'G','H','I','J','K','L',
             'M','O','P','Q','R','S')
Act_names = ('walking','jogging','stairs',
             'sitting','standing','typing',
             'teeth','soup','chips','pasta',
             'drinking','sandwich','kicking',
             'catch','dribbling','writing',
             'clapping','folding')
axis = ('x','y','z')
index_loc = 1

''' 
model building part 
(some need global in function definition)
'''
window = 10
stride = 5
Hz = 20
max_epoch = 10
learnR = 0.001
net = HaiNet().double()
optimizer = optim.Adam(net.parameters(), lr=learnR)

In [ ]:
""" phase_5-1 data loading """
Data = load_data()

In [ ]:
""" phase_5-2 data splitting """
mat = split_data(Data)

In [ ]:
""" phase_5-3 data merging """
matr, target = merge_data(mat)
matrix = np.array(matr)

In [ ]:
train_set, test_set, train_tar, test_tar = divide_data(matrix, target, ratio=0.1)

In [ ]:
debug_set = train_set[3000:4000]
debug_tar = train_tar[3000:4000]

In [ ]:
""" phase_6-1 data training """ 
train(train_set, train_tar)

In [ ]:
def crossloss(output, target):
    global Act_index
    length = len(Act_index)
    tar = np.zeros(length)
    tar[target-1] = 1
    tar = torch.from_numpy(tar).double()
    out = output[0]
    d = torch.dot(out, tar)
    c = - torch.log(d)
    return c
tar = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
tar = np.array(tar)
tar = torch.from_numpy(tar).double()
print(tar)
brr = arr[0]
print(brr)
crr = torch.dot(brr, tar)
print(crr)

In [ ]:
some_exp = torch.zeros(18)

In [ ]:
for i in range(18):
    some_exp[i] = torch.exp(brr[i])

In [ ]:
some_exp

In [ ]:
som = torch.sum(some_exp)

In [ ]:
som

In [ ]:
prob = torch.zeros(18)
for i in range(18):
    prob[i] = some_exp[i] / som

In [ ]:
prob